In [50]:
import warnings
warnings.filterwarnings(action='ignore')

In [51]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf, q_stat
from scipy.stats import skew, kurtosis, shapiro
from hurst import compute_Hc  # For Hurst exponent
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

# Load dataset
df = pd.read_parquet('https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet')
labels = pd.read_csv('/home/pranav-pc/projects/ts/ts/notebooks/learning/statsalgo/evaluation_m4-hourly.csv')['best_model'].astype(str)

# Encode categorical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)  # Convert to numerical labels

# Function to normalize time series using Min-Max Scaling
def normalize_series(series):
    if np.ptp(series) == 0:  # Avoid division by zero
        return np.zeros_like(series)
    scaler = MinMaxScaler()
    series = series.reshape(-1, 1)  # Reshape for sklearn scaler
    return scaler.fit_transform(series).flatten()

# Extract statistical features
def extract_features(time_series):
    features = []
    
    for series in time_series:
        series = np.nan_to_num(series, nan=0.0, posinf=0.0, neginf=0.0)  # Handle NaN and infinite values
        series = normalize_series(series)  # Normalize before feature extraction
        
        mean_val = np.mean(series)
        var_val = np.var(series)
        skew_val = skew(series)
        kurt_val = kurtosis(series)
        total_points = len(series)
        
        if len(series) > 10 and np.all(np.isfinite(series)):
            try:
                adf_pval = max(adfuller(series)[1], 1e-10)
                stationarity = adf_pval < 0.05  # Stationary if p-value < 0.05
            except ValueError:
                adf_pval = np.nan
                stationarity = np.nan
            
            try:
                kpss_pval = max(kpss(series, nlags="auto")[1], 1e-10)
            except ValueError:
                kpss_pval = np.nan
        else:
            adf_pval, kpss_pval, stationarity = np.nan, np.nan, np.nan
        
        try:
            lb_pval = max(q_stat(acf(series, nlags=min(len(series)-1, 10), fft=True)[1:], len(series))[1][-1], 1e-10) if len(series) > 10 else np.nan
        except ValueError:
            lb_pval = np.nan
        
        try:
            shapiro_pval = max(shapiro(series)[1], 1e-10) if len(series) > 10 else np.nan  # Shapiro-Wilk test for normality
        except ValueError:
            shapiro_pval = np.nan
        
        # Additional useful features
        hurst_exp = np.nan  # Default value
        quasi_constant_factor = np.ptp(series) / (np.abs(mean_val) + 1e-6)  # Avoid division by zero
        
        if np.ptp(series) > 1e-6 and quasi_constant_factor > 1e-3:  # Compute Hurst exponent only if series varies significantly
            try:
                hurst_exp, _, _ = compute_Hc(series, kind='change', simplified=True)
            except FloatingPointError:
                hurst_exp = np.nan
        
        acf_values = acf(series, nlags=min(len(series)-1, 10), fft=True)
        pacf_values = pacf(series, nlags=min(len(series)-1, 5))
        
        autocorr_lag1 = acf_values[1] if len(acf_values) > 1 else np.nan
        autocorr_lag5 = acf_values[5] if len(acf_values) > 5 else np.nan
        autocorr_lag10 = acf_values[10] if len(acf_values) > 10 else np.nan
        
        pacf_lag1 = pacf_values[1] if len(pacf_values) > 1 else np.nan
        pacf_lag5 = pacf_values[5] if len(pacf_values) > 5 else np.nan
        
        mad = np.mean(np.abs(series - mean_val))  # Mean Absolute Deviation
        rolling_var = np.var(pd.Series(series).rolling(window=5, min_periods=1).mean())  # Rolling mean variability
        perc_above_median = np.sum(series > np.median(series)) / len(series) if len(series) > 0 else np.nan  # Percentage above median
        
        # Seasonality checks (hourly data)
        daily_seasonality = np.allclose(series[:24], series[-24:], atol=1e-2) if len(series) >= 24 else np.nan
        weekly_seasonality = np.allclose(series[:24*7], series[-24*7:], atol=1e-2) if len(series) >= 24*7 else np.nan
        
        features.append([
            mean_val, var_val, skew_val, kurt_val, total_points, adf_pval, kpss_pval, lb_pval, 
            shapiro_pval, hurst_exp, autocorr_lag1, autocorr_lag5, autocorr_lag10,
            pacf_lag1, pacf_lag5, mad, rolling_var, quasi_constant_factor,
            stationarity, daily_seasonality, weekly_seasonality, perc_above_median
        ])
    
    features_df = pd.DataFrame(features, columns=[
        'Mean', 'Variance', 'Skewness', 'Kurtosis', 'Total_Points', 'ADF_p', 'KPSS_p', 'LB_p', 'Shapiro_p',
        'Hurst_Exp', 'Autocorr_Lag1', 'Autocorr_Lag5', 'Autocorr_Lag10',
        'PACF_Lag1', 'PACF_Lag5', 'MAD', 'Rolling_Var', 'Quasi_Constant_Factor',
        'Stationarity', 'Daily_Seasonality', 'Weekly_Seasonality', 'Perc_Above_Median'
    ])
    
    return features_df.replace([np.inf, -np.inf], np.nan).dropna()

# Convert DataFrame into a format suitable for feature extraction
time_series_data = df.pivot(index='ds', columns='unique_id', values='y').T.values  # Each row is a time series

# Extract features
features_df = extract_features(time_series_data)
print('Feature Extraction done!')

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_df, y_encoded, test_size=0.2, random_state=42)

# Train an XGBoost classifier
clf = xgb.XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss',device="cuda")
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Feature Extraction done!
Accuracy: 0.3855421686746988


ValueError: Number of classes, 8, does not match size of target_names, 9. Try specifying the labels parameter